![Img](https://app.theheadstarter.com/static/hs-logo-opengraph.png)

# Headstarter Financial Analysis & Automation with LLMs Project

# Install Libraries

In [ ]:
! pip install yfinance langchain_pinecone python-dotenv langchain-community sentence_transformers

In [ ]:
from langchain_pinecone import PineconeVectorStore
import dotenv
import json
import yfinance as yf
import concurrent.futures
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata
from langchain.schema import Document
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone
import numpy as np
import requests
import os

In [ ]:
def get_stock_info(symbol: str) -> dict:
    """
    Retrieves and formats detailed information about a stock from Yahoo Finance.

    Args:
        symbol (str): The stock ticker symbol to look up.

    Returns:
        dict: A dictionary containing detailed stock information, including ticker, name,
              business summary, city, state, country, industry, and sector.
    """
    data = yf.Ticker(symbol)
    stock_info = data.info

    properties = {
        "Ticker": stock_info.get('symbol', 'Information not available'),
        'Name': stock_info.get('longName', 'Information not available'),
        'Business Summary': stock_info.get('longBusinessSummary'),
        'City': stock_info.get('city', 'Information not available'),
        'State': stock_info.get('state', 'Information not available'),
        'Country': stock_info.get('country', 'Information not available'),
        'Industry': stock_info.get('industry', 'Information not available'),
        'Sector': stock_info.get('sector', 'Information not available')
    }

    return properties

In [ ]:
data = yf.Ticker("NVDA")
stock_info = data.info
print(stock_info)

{'address1': '2788 San Tomas Expressway', 'city': 'Santa Clara', 'state': 'CA', 'zip': '95051', 'country': 'United States', 'phone': '408 486 2000', 'website': 'https://www.nvidia.com', 'industry': 'Semiconductors', 'industryKey': 'semiconductors', 'industryDisp': 'Semiconductors', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': "NVIDIA Corporation provides graphics and compute and networking solutions in the United States, Taiwan, China, Hong Kong, and internationally. The Graphics segment offers GeForce GPUs for gaming and PCs, the GeForce NOW game streaming service and related infrastructure, and solutions for gaming platforms; Quadro/NVIDIA RTX GPUs for enterprise workstation graphics; virtual GPU or vGPU software for cloud-based visual and virtual computing; automotive platforms for infotainment systems; and Omniverse software for building and operating metaverse and 3D internet applications. The Compute & Networking segment co

In [ ]:
def get_huggingface_embeddings(text, model_name="sentence-transformers/all-mpnet-base-v2"):
    """
    Generates embeddings for the given text using a specified Hugging Face model.

    Args:
        text (str): The input text to generate embeddings for.
        model_name (str): The name of the Hugging Face model to use.
                          Defaults to "sentence-transformers/all-mpnet-base-v2".

    Returns:
        np.ndarray: The generated embeddings as a NumPy array.
    """
    model = SentenceTransformer(model_name)
    return model.encode(text)


def cosine_similarity_between_sentences(sentence1, sentence2):
    """
    Calculates the cosine similarity between two sentences.

    Args:
        sentence1 (str): The first sentence for similarity comparison.
        sentence2 (str): The second sentence for similarity comparison.

    Returns:
        float: The cosine similarity score between the two sentences,
               ranging from -1 (completely opposite) to 1 (identical).

    Notes:
        Prints the similarity score to the console in a formatted string.
    """
    # Get embeddings for both sentences
    embedding1 = np.array(get_huggingface_embeddings(sentence1))
    embedding2 = np.array(get_huggingface_embeddings(sentence2))

    # Reshape embeddings for cosine_similarity function
    embedding1 = embedding1.reshape(1, -1)
    embedding2 = embedding2.reshape(1, -1)

    # Calculate cosine similarity
    similarity = cosine_similarity(embedding1, embedding2)
    similarity_score = similarity[0][0]
    print(f"Cosine similarity between the two sentences: {similarity_score:.4f}")
    return similarity_score


# Example usage
sentence1 = "I like walking to the park"
sentence2 = "I like running to the office"

similarity = cosine_similarity_between_sentences(sentence1, sentence2)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Cosine similarity between the two sentences: 0.6133


In [ ]:
aapl_info = get_stock_info('AAPL')
print(aapl_info)

{'Ticker': 'AAPL', 'Name': 'Apple Inc.', 'Business Summary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts, as well as advertising services include third-party licensing arrangements and its own advertising platforms. In addition, the company offers various subscription-based services, such as Apple Arcade, a game subscription service; Apple Fitness+, a personalized fitness service; Apple Music, which offers users a curated listening experien

In [ ]:
aapl_description = aapl_info['Business Summary']

company_description = "I want to find companies that make smartphones and are headquarted in California"

similarity = cosine_similarity_between_sentences(aapl_description, company_description)

Cosine similarity between the two sentences: 0.3635


# Get all the Stocks in the Stock Market

First, we need to get the symbols (also known as tickers) of all the stocks in the stock market


In [ ]:
def get_company_tickers():
    """
    Downloads and parses the Stock ticker symbols from the GitHub-hosted SEC company tickers JSON file.

    Returns:
        dict: A dictionary containing company tickers and related information.

    Notes:
        The data is sourced from the official SEC website via a GitHub repository:
        https://raw.githubusercontent.com/team-headstart/Financial-Analysis-and-Automation-with-LLMs/main/company_tickers.json
    """
    # URL to fetch the raw JSON file from GitHub
    url = "https://raw.githubusercontent.com/team-headstart/Financial-Analysis-and-Automation-with-LLMs/main/company_tickers.json"

    # Making a GET request to the URL
    response = requests.get(url)

    # Checking if the request was successful
    if response.status_code == 200:
        # Parse the JSON content directly
        company_tickers = json.loads(response.content.decode('utf-8'))

        # Optionally save the content to a local file for future use
        with open("company_tickers.json", "w", encoding="utf-8") as file:
            json.dump(company_tickers, file, indent=4)

        print("File downloaded successfully and saved as 'company_tickers.json'")
        return company_tickers
    else:
        print(f"Failed to download file. Status code: {response.status_code}")
        return None

company_tickers = get_company_tickers()

File downloaded successfully and saved as 'company_tickers.json'


In [ ]:
company_tickers

{'0': {'cik_str': 1045810, 'ticker': 'NVDA', 'title': 'NVIDIA CORP'},
 '1': {'cik_str': 320193, 'ticker': 'AAPL', 'title': 'Apple Inc.'},
 '2': {'cik_str': 789019, 'ticker': 'MSFT', 'title': 'MICROSOFT CORP'},
 '3': {'cik_str': 1018724, 'ticker': 'AMZN', 'title': 'AMAZON COM INC'},
 '4': {'cik_str': 1652044, 'ticker': 'GOOGL', 'title': 'Alphabet Inc.'},
 '5': {'cik_str': 1326801, 'ticker': 'META', 'title': 'Meta Platforms, Inc.'},
 '6': {'cik_str': 1318605, 'ticker': 'TSLA', 'title': 'Tesla, Inc.'},
 '7': {'cik_str': 1067983,
  'ticker': 'BRK-B',
  'title': 'BERKSHIRE HATHAWAY INC'},
 '8': {'cik_str': 1046179,
  'ticker': 'TSM',
  'title': 'TAIWAN SEMICONDUCTOR MANUFACTURING CO LTD'},
 '9': {'cik_str': 1730168, 'ticker': 'AVGO', 'title': 'Broadcom Inc.'},
 '10': {'cik_str': 59478, 'ticker': 'LLY', 'title': 'ELI LILLY & Co'},
 '11': {'cik_str': 19617, 'ticker': 'JPM', 'title': 'JPMORGAN CHASE & CO'},
 '12': {'cik_str': 104169, 'ticker': 'WMT', 'title': 'Walmart Inc.'},
 '13': {'cik_str'

In [ ]:
len(company_tickers)

9998

# Inserting Stocks into Pinecone

**1. Create an account on [Pinecone.io](https://app.pinecone.io/)**

**2. Create a new index called "stocks" and set the dimensions to 768. Leave the rest of the settings as they are.**

![Screenshot 2024-12-02 at 4 48 03 PM](https://github.com/user-attachments/assets/13b484da-cd00-4337-a4db-4779080eb42c)


**3. Create an API Key for Pinecone**

![Screenshot 2024-11-24 at 10 44 37 PM](https://github.com/user-attachments/assets/e7feacc6-2bd1-472a-82e5-659f65624a88)


**4. Store your Pinecone API Key within Google Colab's secrets section, and then enable access to it (see the blue checkmark)**


![Screenshot 2024-11-24 at 10 45 25 PM](https://github.com/user-attachments/assets/eaf73083-0b5f-4d17-9e0c-eab84f91b0bc)




In [ ]:
pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key

index_name = "stocks"
namespace = "stock-descriptions"

hf_embeddings = HuggingFaceEmbeddings()
vectorstore = PineconeVectorStore(index_name=index_name, embedding=hf_embeddings)

<ipython-input-10-8d5c0f356cf6>:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = HuggingFaceEmbeddings()
<ipython-input-10-8d5c0f356cf6>:7: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  hf_embeddings = HuggingFaceEmbeddings()


# Sequential Processing

It will take very long to process all the stocks like this!

[![](https://mermaid.ink/img/pako:eNqNkl1rgzAUhv9KyMXYwF74cSVjYA0rhY6WKRQWe5FqWqU1cUm8GKX_ffmwa65Gc6Hn5LxvzmM8F1jzhsIUHgUZWlCiigG9MlwoXp_AqpPqdS_esmyzCsBivV7o10fxXgagLFbZDsxmb2COi44dzxRsBK-plDoBZSsoaXbuNPeU4941qWBBv0fKVEfOvud5yejh0NWdLr1U0LnMmts2eYhDMLsZgEHa3TV5aEXbEG-zZZnasiXfLMGnaScVeNKRHDiT1DNunRGF5pMFtUaAiCKe5h4hp84jHHks9mJ8mMjBRBOMrT9G45wommis84bjYThZHuPYwzA_xqeIHUU8UZjyYxDOiOIJwhj_uRKnzhOc3FHsdHgoiUNJJhRTfgzFGVEyoRijj0JZAwPYU9GTrtFjfDHbFVQt7WkFUx02RJzMMF21joyKFz-shqkSIw2g4OOxhemBnKXOxqEhiqKO6DHt_3YHwr44v-XXX7It6B4?type=png)](https://mermaid.live/edit#pako:eNqNkl1rgzAUhv9KyMXYwF74cSVjYA0rhY6WKRQWe5FqWqU1cUm8GKX_ffmwa65Gc6Hn5LxvzmM8F1jzhsIUHgUZWlCiigG9MlwoXp_AqpPqdS_esmyzCsBivV7o10fxXgagLFbZDsxmb2COi44dzxRsBK-plDoBZSsoaXbuNPeU4941qWBBv0fKVEfOvud5yejh0NWdLr1U0LnMmts2eYhDMLsZgEHa3TV5aEXbEG-zZZnasiXfLMGnaScVeNKRHDiT1DNunRGF5pMFtUaAiCKe5h4hp84jHHks9mJ8mMjBRBOMrT9G45wommis84bjYThZHuPYwzA_xqeIHUU8UZjyYxDOiOIJwhj_uRKnzhOc3FHsdHgoiUNJJhRTfgzFGVEyoRijj0JZAwPYU9GTrtFjfDHbFVQt7WkFUx02RJzMMF21joyKFz-shqkSIw2g4OOxhemBnKXOxqEhiqKO6DHt_3YHwr44v-XXX7It6B4)

In [ ]:
for idx, stock in company_tickers.items():
    stock_ticker = stock['ticker']
    stock_data = get_stock_info(stock_ticker)
    stock_description = stock_data['Business Summary']

    print(f"Processing stock {idx} / {len(company_tickers)} :", stock_ticker)

    vectorstore_from_documents = PineconeVectorStore.from_documents(
        documents=[Document(page_content=stock_description, metadata=stock_data)],
        embedding=hf_embeddings,
        index_name=index_name,
        namespace=namespace
    )

Processing stock 0 / 9998 : NVDA
Processing stock 1 / 9998 : AAPL
Processing stock 2 / 9998 : MSFT
Processing stock 3 / 9998 : AMZN


KeyboardInterrupt: 

# Parallelizing

[![](https://mermaid.ink/img/pako:eNqFk0uLgzAQgP_KkMOe7MHXRZaC1baXvsDCwqqHrGarVJMSE9hS-983NnWxULceRmf4Pp2MyQVlLCfIQweOTwXsw4SCuvw4Eiw7wqpsxPsXn_r-bmXAcrtdqts6WuwN2Ecr3wB__blJYTKZwixe45JCwKjgrKoI7zwXdphjlVXwwfiR8CbVH9BxdjPbqKxlJTAlTDbVuYXAjDUNZveSHWcZaRromkj_F61etIbire8Xpt2b9tDslvpCdHrRGYrddF6Ibi-6D4vsBjqcUWDe_NCMl0TAG6gfwwmEWOA7FlgasEYBWwP2KOBowBkFXA24Y4COoW61DRklLcwvQUHUHooEFrK53hHrAbkX7WdF51nRfVLUcX4fs8y6ObawMON-phvyI2CGRVakyEA14TUuc7XnL52ZIFGQmiTIU4855scEJfSqOCwFi840Q57gkhiIM3kokPeNq0Zl8pRjQcISq4NT_1VPmH4y1ufXXwdkCM8?type=png)](https://mermaid.live/edit#pako:eNqFk0uLgzAQgP_KkMOe7MHXRZaC1baXvsDCwqqHrGarVJMSE9hS-983NnWxULceRmf4Pp2MyQVlLCfIQweOTwXsw4SCuvw4Eiw7wqpsxPsXn_r-bmXAcrtdqts6WuwN2Ecr3wB__blJYTKZwixe45JCwKjgrKoI7zwXdphjlVXwwfiR8CbVH9BxdjPbqKxlJTAlTDbVuYXAjDUNZveSHWcZaRromkj_F61etIbire8Xpt2b9tDslvpCdHrRGYrddF6Ibi-6D4vsBjqcUWDe_NCMl0TAG6gfwwmEWOA7FlgasEYBWwP2KOBowBkFXA24Y4COoW61DRklLcwvQUHUHooEFrK53hHrAbkX7WdF51nRfVLUcX4fs8y6ObawMON-phvyI2CGRVakyEA14TUuc7XnL52ZIFGQmiTIU4855scEJfSqOCwFi840Q57gkhiIM3kokPeNq0Zl8pRjQcISq4NT_1VPmH4y1ufXXwdkCM8)

In [ ]:
# Initialize tracking lists
successful_tickers = []
unsuccessful_tickers = []

# Load existing successful/unsuccessful tickers
try:
    with open('successful_tickers.txt', 'r') as f:
        successful_tickers = [line.strip() for line in f if line.strip()]
    print(f"Loaded {len(successful_tickers)} successful tickers")
except FileNotFoundError:
    print("No existing successful tickers file found")

try:
    with open('unsuccessful_tickers.txt', 'r') as f:
        unsuccessful_tickers = [line.strip() for line in f if line.strip()]
    print(f"Loaded {len(unsuccessful_tickers)} unsuccessful tickers")
except FileNotFoundError:
    print("No existing unsuccessful tickers file found")

No existing successful tickers file found
No existing unsuccessful tickers file found


In [ ]:
def process_stock(stock_ticker: str) -> str:
    # Skip if already processed
    if stock_ticker in successful_tickers:
        return f"Already processed {stock_ticker}"

    try:
        # Get and store stock data
        stock_data = get_stock_info(stock_ticker)
        stock_description = stock_data['Business Summary']

        # Store stock description in Pinecone
        vectorstore_from_texts = PineconeVectorStore.from_documents(
            documents=[Document(page_content=stock_description, metadata=stock_data)],
            embedding=hf_embeddings,
            index_name=index_name,
            namespace=namespace
        )

        # Track success
        with open('successful_tickers.txt', 'a') as f:
            f.write(f"{stock_ticker}\n")
        successful_tickers.append(stock_ticker)

        return f"Processed {stock_ticker} successfully"

    except Exception as e:
        # Track failure
        with open('unsuccessful_tickers.txt', 'a') as f:
            f.write(f"{stock_ticker}\n")
        unsuccessful_tickers.append(stock_ticker)

        return f"ERROR processing {stock_ticker}: {e}"

def parallel_process_stocks(tickers: list, max_workers: int = 10) -> None:
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_ticker = {
            executor.submit(process_stock, ticker): ticker
            for ticker in tickers
        }

        for future in concurrent.futures.as_completed(future_to_ticker):
            ticker = future_to_ticker[future]
            try:
                result = future.result()
                print(result)

                # Stop on error
                if result.startswith("ERROR"):
                    print(f"Stopping program due to error in {ticker}")
                    executor.shutdown(wait=False)
                    raise SystemExit(1)

            except Exception as exc:
                print(f'{ticker} generated an exception: {exc}')
                print("Stopping program due to exception")
                executor.shutdown(wait=False)
                raise SystemExit(1)

# Prepare your tickers
tickers_to_process = [company_tickers[num]['ticker'] for num in company_tickers.keys()]

# Process them
parallel_process_stocks(tickers_to_process, max_workers=10)

Processed NVDA successfully
Processed MSFT successfully
Processed AAPL successfully
Processed AMZN successfully
Processed AVGO successfully
Processed TSM successfully
Processed META successfully
Processed GOOGL successfully
Processed TSLA successfully
Processed BRK-B successfully
Processed WMT successfully
Processed UNH successfully
Processed V successfully
Processed NVO successfully
Processed SPY successfully
Processed MA successfully
Processed LLY successfully
Processed ORCL successfully
Processed XOM successfully
Processed JPM successfully
Processed HD successfully
Processed COST successfully
Processed PG successfully
Processed RCIT successfully
Processed ABBV successfully
Processed CRM successfully
Processed CVX successfully
Processed JNJ successfully
Processed NFLX successfully
Processed BAC successfully
Processed TMUS successfully
Processed KO successfully
Processed SAP successfully
Processed ASML successfully
Processed WFC successfully
Processed MRK successfully
Processed ADBE s

ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ITT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ITT&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MBLY?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=MBLY&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/HST?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=HST&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance

KeyboardInterrupt: 

# Perform RAG

In [ ]:
# Initialize Pinecone
pc = Pinecone(api_key=userdata.get("PINECONE_API_KEY"))

# Connect to your Pinecone index
pinecone_index = pc.Index(index_name)

ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/KGC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=KGC&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/BEN?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=BEN&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/SJM?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=SJM&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/q

In [ ]:
query = "What are some companies that manufacture consumer hardware?"

In [ ]:
raw_query_embedding = get_huggingface_embeddings(query)

ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/NICE?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=NICE&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/KNSL?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=KNSL&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/TECH?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=TECH&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/fin

In [ ]:
top_matches = pinecone_index.query(vector=raw_query_embedding.tolist(), top_k=10, include_metadata=True, namespace=namespace)

ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AOS?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=AOS&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/EMRAF?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=EMRAF&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/COKE?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=COKE&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/fin

In [ ]:
top_matches

In [ ]:
contexts = [item['metadata']['text'] for item in top_matches['matches']]

ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/CHDN?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=CHDN&crumb=Edge%3A+Too+Many+Requests


In [ ]:
augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query

In [ ]:
print(augmented_query)

<CONTEXT>
Kimball Electronics, Inc. engages in the provision of electronics manufacturing, engineering, and supply chain support services to customers in the automotive, medical, and industrial end markets. The Company also offers contract manufacturing services, including engineering and supply chain support for the production of electronic assemblies and other products, including non-electronic components, medical devices, medical disposables, and precision molded plastics, as well as automation, test, and inspection equipment primarily used in automotive, medical, and industrial applications. In addition, it is also involved in the production and testing of printed circuit board assemblies, as well as clean room assembly, cold chain, and product sterilization management activities. Further, the company offers design services and support, supply chain services and support, rapid prototyping and new product introduction support, product design and process validation and qualification,

ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/PSO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=PSO&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/LKQ?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=LKQ&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/CSXXY?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=CSXXY&crumb=Edge%3A+Too+Many+Requests


# Setting up Groq for RAG

1. Get your Groq API Key [here](https://console.groq.com/keys). Through Groq, you get free access to various LLMs

2. Paste your Groq API Key into your Google Colab secrets, and make sure to enable permissions for it

![Screenshot 2024-11-25 at 12 00 16 AM](https://github.com/user-attachments/assets/e5525d29-bca6-4dbd-892b-cc770a6b281d)

In [ ]:
!pip install openai==0.28
import openai
openai.api_base = "https://api.groq.com/openai/v1"
openai.api_key = userdata.get("GROQ_API_KEY")

system_prompt = f"""You are an expert at providing answers about stocks. Please answer my question provided.
"""

llm_response = openai.ChatCompletion.create(
    model="llama-3.1-70b-versatile",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": augmented_query}
    ]
)

response = llm_response.choices[0].message.content


ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AS?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=AS&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/SNX?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=SNX&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/RBC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=RBC&crumb=Edge%3A+Too+Many+Requests


ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/DTM?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=DTM&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/APG?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=APG&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/FRT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=FRT&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/q

In [ ]:
print(response)

The provided context seems to focus on electronic component distribution and contract manufacturing, rather than companies that manufacture consumer hardware directly. However, some companies in the provided context might work closely with or be suppliers to companies that manufacture consumer hardware.

That being said, here are a few companies that might be indirectly related to the manufacturing of consumer hardware or might work closely with companies that do:

1. Kimball Electronics, Inc. - While they primarily offer contract manufacturing services, they might work with companies that produce consumer hardware, especially in the automotive and medical fields.

2. Arrow Electronics, Inc. - As a distributor of electronic components and provider of enterprise computing solutions, they might work with companies that manufacture consumer hardware, especially in the areas of computing and industrial applications.

3. Avnet, Inc. - Similar to Arrow, Avnet is a distributor of electronic c

ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/RVMD?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=RVMD&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/SARO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=SARO&crumb=Edge%3A+Too+Many+Requests


In [ ]:
# to better improve the response, we could embed functions separately.

Market Firehose

In [ ]:
import requests
from bs4 import BeautifulSoup

r = requests.get("https://www.foxnews.com/us/manhunt-unitedhealthcare-ceo-murder-suspect-begins-5th-day")

soup = BeautifulSoup(r.content, 'html.parser')

s = soup.find('div', class_='entry-content')
content = soup.find_all('p')
print(content)


ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/CFR?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=CFR&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/XP?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=XP&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ITCI?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ITCI&crumb=Edge%3A+Too+Many+Requests


[<p class="copyright">
      This material may not be published, broadcast, rewritten,
      or redistributed. ©2024 FOX News Network, LLC. All rights reserved.
      Quotes displayed in real-time or delayed by at least 15 minutes. Market data provided by
      <a data-omtr-intcmp="topnav_more_factset" href="https://www.factset.com/">Factset</a>. Powered and implemented by
      <a data-omtr-intcmp="topnav_more_factset_digital_solutions" href="https://www.factset.com/solutions/business-needs/digital-solutions">FactSet Digital Solutions</a>.
      <a data-omtr-intcmp="topnav_more_factset_privacy" href="https://www.factset.com/privacy">Legal Statement</a>. Mutual Fund and ETF data provided by
      <a data-omtr-intcmp="topnav_more_refinitive_info" href="https://lipperalpha.refinitiv.com/">Refinitiv Lipper</a>.
    </p>, <p data-v-543741ec="">Joe Cardinale, a former NYPD Commander, details why the suspect left a trail behind and what investigators are doing to find Brian Thompson's killer

In [ ]:
from google.colab import userdata

article_text = " ".join([p.text for p in content])

ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/YMM?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=YMM&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/SIRI?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=SIRI&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/CRS?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=CRS&crumb=Edge%3A+Too+Many+Requests


In [ ]:
article_text

'\n      This material may not be published, broadcast, rewritten,\n      or redistributed. ©2024 FOX News Network, LLC. All rights reserved.\n      Quotes displayed in real-time or delayed by at least 15 minutes. Market data provided by\n      Factset. Powered and implemented by\n      FactSet Digital Solutions.\n      Legal Statement. Mutual Fund and ETF data provided by\n      Refinitiv Lipper.\n     Joe Cardinale, a former NYPD Commander, details why the suspect left a trail behind and what investigators are doing to find Brian Thompson\'s killer. The nationwide manhunt for the gunman suspected of murdering UnitedHealthCare CEO Brian Thompson entered its fifth day Sunday. Police released the latest photos of the suspect late Saturday night, appearing to show the man in the back of a taxi. Police also recovered a backpack they believe is linked to the killer that contained a jacket and Monopoly money, but no gun. Police believe the gunman fled New York City on a bus shortly after th

In [ ]:
prompt = """You are an expert financial news analyst.

Based on the text in the article provided, return a list of tickers that are related to the article.

<article_text>
  {article_text}
</article_text>

Be sure to also provide an explanation as to why each ticker relates to the article.

Return your response in JSON format like this:
{{
  "tickers": ['AAPL', 'MSFT', 'GOOGL'],
  "explanation" : <explanation of why you chose these tickers>,
  "article_title" : <title of the article>,
  "sentiment" : <sentiment of the article>,
  "marketCap": <market cap of the company>
}}

"""

response = openai.ChatCompletion.create(
    model="llama-3.1-70b-versatile",
    messages=[
        {"role": "system", "content": prompt},
        {"role": "user", "content": article_text}
    ],
    response_format={"type": "json_object"}
)

response = response.choices[0].message.content


Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0f0bff9e40>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0f0bff96f0>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0f300a7d00>: Failed to resolve 'api.fastnews.com

ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/FSI?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=FSI&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/FLNT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=FLNT&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AREN?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=AREN&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finan

Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d037ab414e0>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d037ab40070>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))


ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MNY?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=MNY&crumb=Edge%3A+Too+Many+Requests


Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d037af6d180>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0380f45120>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d037b684760>: Failed to resolve 'api.fastnews.com

In [ ]:
print(response)

{
   "tickers" : ["UNH"],
   "explanation" : "The ticker UNH (UnitedHealth Group) is related to the article because it is the company where the victim, Brian Thompson, was the CEO. This news is potentially impactful to the company's stock and operations.",
   "article_title" : "Police release new images of suspect in killing of UnitedHealthCare CEO Brian Thompson",
   "sentiment" : "Neutral",
   "marketCap" : "UNH market cap is approximately $530 billion USD (Please note: market cap can fluctuate and this information may not be up to date)"
}
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0381aa05e0>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))


ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/LICY?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=LICY&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/SHLT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=SHLT&crumb=Edge%3A+Too+Many+Requests


In [ ]:
import http.client, urllib.parse

conn = http.client.HTTPSConnection('api.thenewsapi.com')

params = urllib.parse.urlencode({
    'api_token': 'TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ',
    'categories': 'business,tech',
    'limit': 3,
    })

conn.request('GET', '/v1/news/all?{}'.format(params))

res = conn.getresponse()
data = res.read()

print(data.decode('utf-8'))

Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d037a694a60>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d037a695330>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d03797f6530>: Failed to resolve 'api.fastnews.com

ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/BTA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=BTA&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/NAZ?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=NAZ&crumb=Edge%3A+Too+Many+Requests


Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d038096b940>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d038121f130>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d037ba08430>: Failed to resolve 'api.fastnews.com

ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/GGR?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=GGR&crumb=Edge%3A+Too+Many+Requests


Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d037a6455a0>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d037a645840>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))

Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d037a83a740>: Failed to resolve 'api.fastnews.com

...

Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d037baf7310>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d037baf7b20>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d03797f7670>: Failed to resolve 'api.fastnews.com

ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/DCF?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=DCF&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MX?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=MX&crumb=Edge%3A+Too+Many+Requests


Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d03807fbee0>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d040453ccd0>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d03807fb850>: Failed to resolve 'api.fastnews.com

ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MHH?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=MHH&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MPAA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=MPAA&crumb=Edge%3A+Too+Many+Requests


Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0379147760>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0379146fb0>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
{"meta":{"found":11369806,"returned":3,"limit":3,"page":1},"data":[{"uuid":"cc3db7c1-0038-4495-b5c5-d8f8f1f4533c","title":"속보) 이상민 사의, 윤석열이 수용 : 클리앙","description":"한동훈 \"사실상 직무배제\"","keywords":"","snippet":"공지\n\n[점검] 잠시후 19:00시 부터 22:00까지 서버 작업으로 인한 서비스 단절이 발생됩니다.","url":"https:\/\/www.clien.net\/serv

In [ ]:
data

b'{"meta":{"found":11369806,"returned":3,"limit":3,"page":1},"data":[{"uuid":"cc3db7c1-0038-4495-b5c5-d8f8f1f4533c","title":"\xec\x86\x8d\xeb\xb3\xb4) \xec\x9d\xb4\xec\x83\x81\xeb\xaf\xbc \xec\x82\xac\xec\x9d\x98, \xec\x9c\xa4\xec\x84\x9d\xec\x97\xb4\xec\x9d\xb4 \xec\x88\x98\xec\x9a\xa9 : \xed\x81\xb4\xeb\xa6\xac\xec\x95\x99","description":"\xed\x95\x9c\xeb\x8f\x99\xed\x9b\x88 \\"\xec\x82\xac\xec\x8b\xa4\xec\x83\x81 \xec\xa7\x81\xeb\xac\xb4\xeb\xb0\xb0\xec\xa0\x9c\\"","keywords":"","snippet":"\xea\xb3\xb5\xec\xa7\x80\\n\\n[\xec\xa0\x90\xea\xb2\x80] \xec\x9e\xa0\xec\x8b\x9c\xed\x9b\x84 19:00\xec\x8b\x9c \xeb\xb6\x80\xed\x84\xb0 22:00\xea\xb9\x8c\xec\xa7\x80 \xec\x84\x9c\xeb\xb2\x84 \xec\x9e\x91\xec\x97\x85\xec\x9c\xbc\xeb\xa1\x9c \xec\x9d\xb8\xed\x95\x9c \xec\x84\x9c\xeb\xb9\x84\xec\x8a\xa4 \xeb\x8b\xa8\xec\xa0\x88\xec\x9d\xb4 \xeb\xb0\x9c\xec\x83\x9d\xeb\x90\xa9\xeb\x8b\x88\xeb\x8b\xa4.","url":"https:\\/\\/www.clien.net\\/service\\/board\\/park\\/18864597","image_url":"https:\\/\\/www.

Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0379962f80>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0379961fc0>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d03800f5840>: Failed to resolve 'api.fastnews.com

In [ ]:
import requests
import openai

def get_fastnews_articles(api_token, num_articles=10):
    """
    Fetches a specified number of articles from the FastNews API.

    Args:
        api_token (str): API token for authentication with the FastNews API.
        num_articles (int): Number of articles to retrieve.

    Returns:
        list: A list of article data dictionaries or an empty list if an error occurs.
    """
    url = "https://api.thenewsapi.com/v1/news/top"  # Replace with the actual FastNews API endpoint
    params = {
        "api_token": api_token,
        "limit": num_articles
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an exception for HTTP errors
        data = response.json()

        if 'data' in data:
            return data['data']
        else:
            print(f"Unexpected response format: {data}")
            return []
    except requests.exceptions.RequestException as e:
        print(f"Error fetching articles: {e}")
        return []

def answer_user_question(user_input, articles, model="gpt-3.5-turbo"):
    """
    Answers a user's question based on the provided articles using OpenAI's API.

    Args:
        user_input (str): The user's question.
        articles (list): A list of articles to base the response on.
        model (str): The OpenAI model to use for generating responses.

    Returns:
        str: The generated response to the user's question.
    """
    # Construct the context string from article content
    context = "\n".join(
        f"{article.get('title', 'No Title')}: {article.get('description', 'No Description')}"
        for article in articles
    )

    # Construct the prompt for the language model
    prompt = f"""
    Answer the following question based on the provided articles:

    Question: {user_input}

    Articles:
    {context}
    """

    try:
        llm_response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ]
        )
        return llm_response.choices[0].message.content
    except openai.error.OpenAIError as e:
        print(f"Error calling OpenAI API: {e}")
        return "An error occurred while processing your request."

if __name__ == "__main__":
    # Replace with your actual API token
    FASTNEWS_API_TOKEN = "TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ"

    # Fetch articles
    articles = get_fastnews_articles(api_token=FASTNEWS_API_TOKEN)

    if articles:
        print("Successfully retrieved articles.")
        user_question = input("Ask a question about the news: ")
        answer = answer_user_question(user_question, articles)
        print("\nResponse:")
        print(answer)
    else:
        print("No articles retrieved. Please check your API token or network connection.")


Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d03800af880>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0380713130>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0f3015bdc0>: Failed to resolve 'api.fastnews.com

ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/TSBK?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=TSBK&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MCR?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=MCR&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/WRN?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=WRN&crumb=Edge%3A+Too+Many+Requests


Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d03799cafb0>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d03799c9d50>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d03799cba90>: Failed to resolve 'api.fastnews.com

ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/BLFY?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=BLFY&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MMT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=MMT&crumb=Edge%3A+Too+Many+Requests


Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d04045d29b0>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d04045d28c0>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0f3012b3a0>: Failed to resolve 'api.fastnews.com

...

ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ACB?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ACB&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ASUR?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ASUR&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MFM?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=MFM&crumb=Edge%3A+Too+Many+Requests


Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d037be249d0>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d040449f460>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0380d829e0>: Failed to resolve 'api.fastnews.com

ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MUE?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=MUE&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MVT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=MVT&crumb=Edge%3A+Too+Many+Requests


Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0379979ff0>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d038094ffa0>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d037b2a0100>: Failed to resolve 'api.fastnews.com

KeyboardInterrupt: Interrupted by user

Streamlit

In [ ]:
! pip install streamlit pyngrok python-dotenv

Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d03710c0430>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d03710c1420>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0370c44dc0>: Failed to resolve 'api.fastnews.com

ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/CNCT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=CNCT&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/GMBL?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=GMBL&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/FKST?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=FKST&crumb=Edge%3A+Too+Many+Requests


Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0370ecbac0>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0370ecad70>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0370ec9fc0>: Failed to resolve 'api.fastnews.com

ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/CDSG?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=CDSG&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/IWAL?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=IWAL&crumb=Edge%3A+Too+Many+Requests


Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d037b5de6e0>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d037b5df4f0>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d03710c0790>: Failed to resolve 'api.fastnews.com

...

Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0380a862f0>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0f30169e10>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d03713726e0>: Failed to resolve 'api.fastnews.com

ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/IGEN?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=IGEN&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/RGIN?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=RGIN&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/GYGC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=GYGC&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/fin

Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0404609660>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d040460a890>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d037a705f90>: Failed to resolve 'api.fastnews.com

ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/APSI?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=APSI&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ALPP?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ALPP&crumb=Edge%3A+Too+Many+Requests


Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0380a84af0>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0ea6d8fa30>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0ea6d8edd0>: Failed to resolve 'api.fastnews.com

In [ ]:
from threading import Thread
from pyngrok import ngrok
from google.colab import userdata

ngrok_token = userdata.get('NGROK_AUTH_TOKEN')
ngrok.set_auth_token(ngrok_token)

ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/HPCO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=HPCO&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/CLCS?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=CLCS&crumb=Edge%3A+Too+Many+Requests


Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0378585fc0>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0378586920>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0378586230>: Failed to resolve 'api.fastnews.com

ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/NEPTF?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=NEPTF&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/WTII?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=WTII&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/WARM?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=WARM&crumb=Edge%3A+Too+Many+Requests


Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0371e29d50>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0371435ba0>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0378585d20>: Failed to resolve 'api.fastnews.com

In [ ]:
def run_streamlit():
  os.system('streamlit run /content/app.py -- server.port 5000')


Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0370d7efe0>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0370a05c90>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0378f8bca0>: Failed to resolve 'api.fastnews.com

In [ ]:
%%writefile app.py

import streamlit as st
import openai
import numpy as np
import os
from google.colab import userdata
from dotenv import load_dotenv
import os

load_dotenv()

# Main app UI
st.write("Quantbot")

Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0370e01390>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0370ecefb0>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0379321090>: Failed to resolve 'api.fastnews.com

In [ ]:
thread = Thread(target = run_streamlit)
thread.start()

Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d03800e8b20>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d03800ea2c0>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0381ce0850>: Failed to resolve 'api.fastnews.com

In [ ]:
public_url = ngrok.connect(addr='5000', proto='http', bind_tls=True)
print("Public URL: ", public_url)

Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0381ce1780>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0f3010b370>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))


Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d037a8fa140>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d0f30108ee0>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d037bfe0280>: Failed to resolve 'api.fastnews.com

PyngrokNgrokHTTPError: ngrok client exception, API returned 502: {"error_code":103,"status_code":502,"msg":"failed to start tunnel","details":{"err":"failed to start tunnel: Your account may not run more than 3 tunnels over a single ngrok agent session.\nThe tunnels already running on this session are:\ntn_2pwOHBhDUR4ZyDbYWyvArfWNObU, tn_2pwOM0esbx9kF99MSzMfDvdG7yF, tn_2pwOEMJs8pcseuOvWaVBNE1DisJ\n\r\n\r\nERR_NGROK_324\r\n"}}


Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d037b7211e0>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
Error fetching articles: HTTPSConnectionPool(host='api.fastnews.com', port=443): Max retries exceeded with url: /v1/articles?api_key=TFp9I8aFM42JBthMLpKdWNbRluZQ25rfQH8ttenQ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d037b720490>: Failed to resolve 'api.fastnews.com' ([Errno -2] Name or service not known)"))
